In [1]:
import numpy as np
import PIL
import pickle
import time
from PIL import Image
from joblib import load
from queue import Queue
from threading import Thread

In [2]:
class ImageMosaicApp :
    
    def __init__(self) :
        self.tileSize = 16
        self.maxThreads = 50
        self.model = load('../bin/model.joblib')
        with open('../data/dataset.pickle', 'rb') as handle:
            self.data = pickle.load(handle)
    
    def getMaze(self, imgPath, imgGenSavePath) : 
        start_time = time.time()
        resizedImg = self.__resize(imgPath)
        expctedQueueSize = (self.cols/self.tileSize) * (self.rows/self.tileSize)
        queue = Queue(maxsize=0)
        self.__imgTileIndexesGenerator(queue, resizedImg)
        imgGen = np.ndarray(shape=(self.rows, self.cols, 3), dtype=np.int8)
        
        for i in range(self.maxThreads) :
            worker = Thread(target=self.__runTasksInQueue, args=(queue, resizedImg, imgGen))
            worker.setDaemon(True) 
            worker.start()
        queue.join()
        
        imgPil = Image.fromarray(imgGen, 'RGB')
        imgPil.save(imgGenSavePath)
        print("Image Maze generated and saved")
        print("--- {} tiles replaced in {} seconds ---".format(expctedQueueSize, time.time() - start_time))
    
    def __imgTileIndexesGenerator(self, queue, imgArr) :
        for i in range(0, self.rows, self.tileSize) :
            for j in range(0, self.cols, self.tileSize) :
                queue.put((i, j))
    
    def __runTasksInQueue(self, queue, imgArr, imgGen) :
        while not queue.empty() :
            indexes = queue.get()
            self.__replaceChunk(indexes[0], indexes[1], imgArr, imgGen)
            queue.task_done()
    
    def __replaceChunk(self, row, col, imgArr, imgGen) :
        chunk = imgArr[:, row:row+self.tileSize, col:col+self.tileSize].flatten()
        nnIndex = self.model.kneighbors(np.expand_dims(chunk, axis=0), return_distance=False)
        imgGen[row:row+self.tileSize, col:col+self.tileSize, :] = np.transpose(np.reshape(self.data[nnIndex[0][0]], (3, self.tileSize, self.tileSize)), (1, 2, 0))
    
    def __resize(self, imgPath) :
        img = Image.open(imgPath)
        self.cols = int(img.width / self.tileSize) * self.tileSize
        self.rows = int(img.height / self.tileSize) * self.tileSize
        img = img.resize((self.cols, self.rows))
        imgData = np.array(img)
        imgData = imgData[...,:3]
        return np.transpose(imgData, (2, 0, 1))
        

In [3]:
mazeApp = ImageMosaicApp()
print("model ready")

model ready


In [6]:
mazeApp.getMaze('../data/testImg1.jpg', '../data/testImgGen1.jpg')
mazeApp.getMaze('../data/testImg2.jpg', '../data/testImgGen2.jpg')
mazeApp.getMaze('../data/testImg3.jpg', '../data/testImgGen3.jpg')
mazeApp.getMaze('../data/testImg4.jpg', '../data/testImgGen4.jpg')
mazeApp.getMaze('../data/testImg5.jpg', '../data/testImgGen5.jpg')
mazeApp.getMaze('../data/testImg6.jpeg', '../data/testImgGen6.jpeg')
mazeApp.getMaze('../data/testImg7.jpg', '../data/testImgGen7.jpg')

Image Maze generated and saved
--- 2480.0 tiles replaced in 90.13449192047119 seconds ---
Image Maze generated and saved
--- 4704.0 tiles replaced in 166.65243220329285 seconds ---
Image Maze generated and saved
--- 1500.0 tiles replaced in 60.01042175292969 seconds ---
Image Maze generated and saved
--- 2074.0 tiles replaced in 79.38735437393188 seconds ---
Image Maze generated and saved
--- 6300.0 tiles replaced in 226.35945916175842 seconds ---
Image Maze generated and saved
--- 187.0 tiles replaced in 6.674035549163818 seconds ---
Image Maze generated and saved
--- 2464.0 tiles replaced in 101.95076966285706 seconds ---
